In [ ]:
import os 
os.chdir("/workspaces/p99-bluesky/tests/")

import asyncio
import subprocess, os
from bluesky.run_engine import RunEngine
from ophyd_async.core import DeviceCollector
from ophyd_async.epics.signal import epics_signal_r, epics_signal_rw
from ophyd_async.core.signal import observe_value, wait_for_value
from p99_bluesky.devices.p99.sample_stage import (
    FilterMotor,
    SampleAngleStage,
    p99StageSelections,
)
from p99_bluesky.plans.fast_scan import fast_scan_1d, fast_scan_grid
from p99_bluesky.plans.stxm import stxm_fast,get_velocity_and_step_size
from soft_motor import SoftThreeAxisStage
from bluesky.run_engine import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.plans import count, scan 
from ophyd.sim import noisy_det as det
bec = BestEffortCallback()
RE = RunEngine({})
from databroker import Broker
db = Broker.named('temp')
RE.subscribe(db.insert)
with DeviceCollector(mock=False):
    mock_sampleAngleStage = SampleAngleStage(
        "p99-MO-TABLE-01:", name="mock_sampleAngleStage"
    )
    mock_filter_wheel = FilterMotor(
        "p99-MO-STAGE-02:MP:SELECT", name="mock_filter_wheel"
    )
    xyz_motor = SoftThreeAxisStage("p99-MO-STAGE-02:", name="xyz_motor")

In [ ]:
uid, = RE(stxm_fast(
    det=det,
    count_time = 0.5,
    scan_motor = xyz_motor.x,
    scan_start = -2.2,
    scan_end   = 2.2,
    step_motor = xyz_motor.y,
    step_start = -2.2,
    step_end   = 2.2,
    plan_time  = 50,
),bec)

In [ ]:
header = db[-1]
data = header.table()
plt.tricontourf(data["xyz_motor-y"],data["xyz_motor-x"],data['noisy_det'])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
k = RE(get_velocity_and_step_size(xyz_motor.x, 20, 0.1))

In [ ]:
RE.abort() 

In [ ]:
RE(fast_scan_grid([det],xyz_motor.x,-2,2,3, xyz_motor.y,-0,-10,1),bec)

In [ ]:
RE(fast_scan_grid([det],xyz_motor.x,0,5,10, xyz_motor.y,-0,-10,5,snake_axes = True),bec)

In [ ]:
RE(fast_scan([det],xyz_motor.x,-8.888,18880,5
            ))

In [ ]:
import bluesky.plan_patterns as bpp
import bluesky.plan_stubs as bps
from bluesky.simulators import check_limits
from ophyd_async.epics.motion import Motor
from ophyd_async.protocols import AsyncReadable
import asyncio
from typing import Optional


In [ ]:
def plan(motor):
    y = yield from bps.read(motor.x)
    print(y)

In [ ]:
RE(bps.abs_set(xyz_motor.y.velocity,100))
   

In [ ]:
RE(bps.abs_set(xyz_motor.x.velocity,100))

In [ ]:
def set_and_count(det,motor,end):
    yield from bps.abs_set(xyz_motor.x,end) 
    current_value = yield from bps.rd(motor.user_readback)
    while abs(end - current_value) > 1e-7:
        yield from count(det+[motor], num=1)
        current_value = yield from bps.rd(motor.user_readback)

In [ ]:
from ophyd.sim import det, det1, motor1
from bluesky.plans import count
from bluesky.preprocessors import monitor_during_wrapper

# Record any updates from det1 while 'counting' det 5 times.
RE(set_and_count([det],xyz_motor.x,7),bec)

In [ ]:
a = [1,2,3,4]
b = 5

In [ ]:
c = a,b

In [ ]:
c

In [ ]:
from bluesky.plans import grid_scan
from ophyd.sim import det5, jittery_motor1, jittery_motor2
from bluesky.callbacks.mpl_plotting import LiveScatter

# The 'jittery' example motors won't go exactly where they are told to go.

RE(grid_scan([det5],
                      jittery_motor1, -3, 3, 6,
                      jittery_motor2, -5, 5, 10, False),
   LiveScatter('jittery_motor1', 'jittery_motor2', 'det5',
            xlim=(-3, 3), ylim=(-5, 5)))

In [ ]:
def test(count_time: float,
    scan_range: float,
    step_range: float,
    plan_time :float,):
    num_data_point = plan_time/count_time
    step_size = 1.0/(num_data_point/(scan_range*step_range))**0.5
    ideal_velocity = step_size/count_time
    #smallest_possible_step_size = scan_motor.max_velocity

    print(ideal_velocity,step_size)

In [ ]:
test(0.1,10,2,100)

In [ ]:
(100/0.1/20)**0.5

In [ ]:
(7.07*2) *(7.07*10)
